# Software Version Notice

This script is developed under Python 3.6 and Gurobi 8.1.

Obtain licenses from [Gurobi Download Center](http://www.gurobi.com/downloads/download-center).

In [1]:
# Show Python version
import sys
print('Python version:', sys.version)

Python version: 3.6.5 |Anaconda, Inc.| (default, Mar 29 2018, 13:32:41) [MSC v.1900 64 bit (AMD64)]


In [2]:
import gurobipy as grb
print('Gurobi version:', grb.GRB.VERSION_MAJOR, grb.GRB.VERSION_MINOR, grb.GRB.VERSION_TECHNICAL)

Gurobi version: 8 1 1


In [3]:
import os
import pandas as pd
from LH_MODULE_CONFIG import SCRATCH_FOLDER, SCENARIO_ROOT
sys.path.insert(0, SCENARIO_ROOT)
from SCENARIO_CONFIG import VEH_FULL_RANGE, VEH_START_RANGE

Active scenario: Baseline


<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Summary" data-toc-modified-id="Summary-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Summary</a></span><ul class="toc-item"><li><span><a href="#Inputs" data-toc-modified-id="Inputs-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Inputs</a></span></li><li><span><a href="#Outputs" data-toc-modified-id="Outputs-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Outputs</a></span></li></ul></li><li><span><a href="#Load-Data" data-toc-modified-id="Load-Data-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Load Data</a></span><ul class="toc-item"><li><span><a href="#Distances-between-nodes" data-toc-modified-id="Distances-between-nodes-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Distances between nodes</a></span></li><li><span><a href="#List-of-all-node-ids" data-toc-modified-id="List-of-all-node-ids-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>List of all node ids</a></span></li><li><span><a href="#Paths-as-sequence-of-nodes" data-toc-modified-id="Paths-as-sequence-of-nodes-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Paths as sequence of nodes</a></span></li></ul></li><li><span><a href="#Gurobi-Optimization-Model" data-toc-modified-id="Gurobi-Optimization-Model-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Gurobi Optimization Model</a></span><ul class="toc-item"><li><span><a href="#Initialize-Gurobi" data-toc-modified-id="Initialize-Gurobi-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Initialize Gurobi</a></span></li><li><span><a href="#Decision-variables" data-toc-modified-id="Decision-variables-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Decision variables</a></span></li><li><span><a href="#Constraints" data-toc-modified-id="Constraints-3.3"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>Constraints</a></span></li><li><span><a href="#Objective" data-toc-modified-id="Objective-3.4"><span class="toc-item-num">3.4&nbsp;&nbsp;</span>Objective</a></span></li><li><span><a href="#Review-Gurobi-model" data-toc-modified-id="Review-Gurobi-model-3.5"><span class="toc-item-num">3.5&nbsp;&nbsp;</span>Review Gurobi model</a></span></li><li><span><a href="#Optimize" data-toc-modified-id="Optimize-3.6"><span class="toc-item-num">3.6&nbsp;&nbsp;</span>Optimize</a></span></li></ul></li><li><span><a href="#Chosen-HRS-Nodes" data-toc-modified-id="Chosen-HRS-Nodes-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Chosen HRS Nodes</a></span></li><li><span><a href="#Find-which-HRS's-refuel-which-paths" data-toc-modified-id="Find-which-HRS's-refuel-which-paths-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Find which HRS's refuel which paths</a></span></li></ul></div>

## Summary
### Inputs
- Node-node distance: `SCRATCH_FOLDER/NodeNode_Dists.csv`.
    Schema: | from_node_id | to_node_id | dist_km |
- Paths, as sequences of nodes: `SCRATCH_FOLDER/Paths.csv`
    Schema: | path_id | orig_node_id | dest_node_id | node_seq | node_id |

### Outputs
- List of nodes chosen for refueling stations: `SCRATCH_FOLDER/Chosen_HRS_nodes.csv`
- Refueling demand for each path at each refueling node (in terms of km of fuel needed): `SCRATCH_FOLDER/Paths_fuel_km.csv`
    Schema: | path_id | refuel_node_id | refuel_km |

## Load Data

### Distances between nodes

Distances between nodes are pre-calculated and saved in `SCRATCH_FOLDER/NodeNode_Dists.csv`

In [4]:
nodenode_dist_df = pd.read_csv(os.path.join(SCRATCH_FOLDER, 'NodeNode_Dists.csv'))
display(nodenode_dist_df.head())

,from_node_id,to_node_id,dist_km
0,1,1,0
1,2,1,48
2,3,1,201
3,4,1,227
4,5,1,242


In [5]:
nodenode_dist = nodenode_dist_df.set_index(['from_node_id', 'to_node_id']).to_dict(orient='index')
nodenode_dist = {k: v['dist_km'] for k, v in nodenode_dist.items()}
# nodenode_dist: {(from_node_id, to_node_id): dist_km}

In [6]:
#print(nodenode_dist)

### List of all node ids

In [7]:
all_node_ids = nodenode_dist_df.from_node_id.unique().tolist()

In [8]:
print(all_node_ids)

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76]


### Paths as sequence of nodes

In [9]:
paths_nodes_df = pd.read_csv(os.path.join(SCRATCH_FOLDER, 'Paths.csv'))
display(paths_nodes_df.head())

,path_id,orig_node_id,dest_node_id,node_seq,node_id
0,0,13,18,0,13
1,0,13,18,1,18
2,1,18,13,0,18
3,1,18,13,1,13
4,2,12,13,0,12


In [10]:
# paths_nodes = {path_id: [node1_id, node2_id, ...]}
# a dictionary that stores the node sequence of every path
paths_nodes = paths_nodes_df\
                .sort_values(by=['path_id', 'node_seq'])\
                [['path_id', 'node_id']]\
                .groupby('path_id')\
                ['node_id']\
                .apply(list)\
                .to_dict()

In [11]:
print(paths_nodes)

{0: [13, 18], 1: [18, 13], 2: [12, 13], 3: [12, 13, 18], 4: [25, 54, 53, 52, 24, 51, 23, 50, 19, 20, 18, 13], 5: [25, 54, 53, 52, 24, 51, 23, 50, 19, 20, 18], 6: [29, 28, 27, 68, 67, 22, 48, 47, 46, 45, 44, 43, 21, 42, 17, 18, 13], 7: [29, 28, 27, 68, 67, 22, 48, 47, 46, 45, 44, 43, 21, 42, 17, 18], 8: [31, 32, 34, 35, 38, 27, 68, 67, 22, 48, 47, 46, 45, 44, 43, 21, 42, 17, 18, 13], 9: [31, 32, 34, 35, 38, 27, 68, 67, 22, 48, 47, 46, 45, 44, 43, 21, 42, 17, 18], 10: [35, 38, 27, 68, 67, 22, 48, 47, 46, 45, 44, 43, 21, 42, 17, 18, 13], 11: [35, 38, 27, 68, 67, 22, 48, 47, 46, 45, 44, 43, 21, 42, 17, 18], 12: [33, 34, 35, 38, 27, 68, 67, 22, 48, 47, 46, 45, 44, 43, 21, 42, 17, 18, 13], 13: [33, 34, 35, 38, 27, 68, 67, 22, 48, 47, 46, 45, 44, 43, 21, 42, 17, 18], 14: [7, 6, 10, 11, 40, 12, 13], 15: [7, 6, 10, 11, 40, 12, 13, 18], 16: [5, 4, 10, 11, 40, 12, 13], 17: [5, 4, 10, 11, 40, 12, 13, 18], 18: [6, 10, 11, 40, 12, 13], 19: [6, 10, 11, 40, 12, 13, 18], 20: [37, 36, 34, 35, 38, 27, 68

In [12]:
all_path_ids = paths_nodes_df['path_id'].unique().tolist()

In [13]:
print(all_path_ids)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221,

## Gurobi Optimization Model

### Initialize Gurobi

In [14]:
import gurobipy as grb

In [37]:
m = grb.Model('CA_LH_FCEV_FRLM')    # FRLM = Flow Refeuling Location Model

### Decision variables

In [38]:
# hrs_at[i] : whether to put a hydrogen refueling station (HRS) at node i
hrs_at = m.addVars(all_node_ids, 
                   vtype=grb.GRB.BINARY, 
                   name='hrs_at')

### Constraints

On each path, each node requires a set of other nodes, to refuel at at least one of them.

Here we construct those sets of 'other nodes' for each `(path_id, node_id)` combination.

In [39]:
from collections import defaultdict

path_node_need_refuel_at = defaultdict(list)
# { (path_id, dest_node_id): [refuel_node_id, refuel_node_id, ...] }
# This dict describes on path <path_id>, which refueling locations can enable travel to node <dest_node_id>

# Criteria for coverage:
# 1) dest_node is downstream of refuel_node
# 2) distance from refuel_node to dest_node is no more than VEH_FULL_RANGE

In [40]:
for path_id, path_nodes in paths_nodes.items():
    orig_node_id = path_nodes[0]
    n_nodes = len(path_nodes)
    for refuel_node_idx, refuel_node_id in enumerate(path_nodes):
        for dest_node_idx in range(refuel_node_idx, n_nodes):
            dest_node_id = path_nodes[dest_node_idx]
            if dest_node_idx <= refuel_node_idx: continue
            if nodenode_dist[orig_node_id, dest_node_id] <= VEH_START_RANGE: continue
            if nodenode_dist[refuel_node_id, dest_node_id] >= VEH_FULL_RANGE: break
            path_node_need_refuel_at[path_id, dest_node_id].append(refuel_node_id)

In [41]:
print(path_node_need_refuel_at)

defaultdict(<class 'list'>, {(4, 23): [25, 54, 53, 52, 24, 51], (4, 50): [25, 54, 53, 52, 24, 51, 23], (4, 19): [25, 54, 53, 52, 24, 51, 23, 50], (4, 20): [25, 54, 53, 52, 24, 51, 23, 50, 19], (4, 18): [24, 51, 23, 50, 19, 20], (4, 13): [51, 23, 50, 19, 20, 18], (5, 23): [25, 54, 53, 52, 24, 51], (5, 50): [25, 54, 53, 52, 24, 51, 23], (5, 19): [25, 54, 53, 52, 24, 51, 23, 50], (5, 20): [25, 54, 53, 52, 24, 51, 23, 50, 19], (5, 18): [24, 51, 23, 50, 19, 20], (6, 22): [29, 28, 27, 68, 67], (6, 48): [29, 28, 27, 68, 67, 22], (6, 47): [29, 28, 27, 68, 67, 22, 48], (6, 46): [68, 67, 22, 48, 47], (6, 45): [68, 67, 22, 48, 47, 46], (6, 44): [67, 22, 48, 47, 46, 45], (6, 43): [22, 48, 47, 46, 45, 44], (6, 21): [48, 47, 46, 45, 44, 43], (6, 42): [46, 45, 44, 43, 21], (6, 17): [44, 43, 21, 42], (6, 18): [43, 21, 42, 17], (6, 13): [43, 21, 42, 17, 18], (7, 22): [29, 28, 27, 68, 67], (7, 48): [29, 28, 27, 68, 67, 22], (7, 47): [29, 28, 27, 68, 67, 22, 48], (7, 46): [68, 67, 22, 48, 47], (7, 45): [

In [42]:
for (path_id, dest_node_id), refuel_node_ids in path_node_need_refuel_at.items():
    m.addConstr(grb.quicksum(hrs_at[f] for f in refuel_node_ids)>=1, 
                name='On path {} reach node {}'.format(path_id, dest_node_id))

### Objective

In [43]:
m.setObjective(grb.quicksum(hrs_at[node_id] for node_id in all_node_ids), sense=grb.GRB.MINIMIZE)

### Review Gurobi model

In [44]:
m.update()

In [45]:
grb_vars = m.getVars()
grb_cons = m.getConstrs()

print('Gurobi Model Summary:')
print('{} decision variables.'.format(len(grb_vars)))
# print([v.VarName for v in grb_vars])
print('{} constraints.'.format(len(grb_cons)))
# print([m.getRow(c) for c in grb_cons])

Gurobi Model Summary:
75 decision variables.
3332 constraints.


### Optimize

In [46]:
%%time
m.optimize()

Optimize a model with 3332 rows, 75 columns and 20080 nonzeros
Variable types: 0 continuous, 75 integer (75 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 14.0000000
Presolve removed 3332 rows and 75 columns
Presolve time: 0.01s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.04 seconds
Thread count was 1 (of 8 available processors)

Solution count 2: 13 14 

Optimal solution found (tolerance 1.00e-04)
Best objective 1.300000000000e+01, best bound 1.300000000000e+01, gap 0.0000%
Wall time: 40 ms


## Chosen HRS Nodes

In [57]:
chosen_hrs_node_id = [node_id for node_id in all_node_ids if hrs_at[node_id].x>0 ]
print(chosen_hrs_node_id)
print(len(chosen_hrs_node_id))

[4, 11, 17, 18, 19, 33, 34, 44, 51, 63, 67, 70, 72]
13


Output chosen HRS sites as a csv

In [48]:
pd.Series(chosen_hrs_node_id).rename('HRS_node_id').to_csv(os.path.join(SCRATCH_FOLDER, 'Chosen_HRS_nodes.csv'), index=False, header=True)

## Find which HRS's refuel which paths

In [49]:
on_path_refuel_at = {}
# {path_id: [refuel_at_node_id, refuel_at_node_id, ...]}
# For each path_id, list all nodes it needs to refuel at

In [50]:
path_refuel_km = {}

In [51]:
for path_id, path_nodes in paths_nodes.items():
    first_refuel = True
    
    for node in path_nodes:
        if node in chosen_hrs_node_id:
            if first_refuel:
                on_path_refuel_at[path_id] = [node]
                path_refuel_km[path_id, node] = VEH_FULL_RANGE - VEH_START_RANGE + nodenode_dist[path_nodes[0], node]
                first_refuel = False
            else:
                prev_refuel_at = on_path_refuel_at[path_id][-1]
                on_path_refuel_at[path_id].append(node)
                path_refuel_km[path_id, node] = nodenode_dist[prev_refuel_at, node]
        
    if path_id not in on_path_refuel_at:
        # This can occur when a path contains no HRS node at all.
        # which means the path needs no refuel. (short path)
        on_path_refuel_at[path_id] = []
    
    print('Path {}:\n\t{} nodes: {} \n\t{} refuels: {}'.format(path_id, 
                                                               len(path_nodes), 
                                                               str(path_nodes),
                                                               len(on_path_refuel_at[path_id]),
                                                               list(zip(on_path_refuel_at[path_id],
                                                                        [path_refuel_km[path_id, refuel_at] 
                                                                         for refuel_at in on_path_refuel_at[path_id]
                                                                        ]
                                                                       )
                                                                   )
                                                              )
         )


Path 0:
	2 nodes: [13, 18] 
	1 refuels: [(18, 145.0)]
Path 1:
	2 nodes: [18, 13] 
	1 refuels: [(18, 120.0)]
Path 2:
	2 nodes: [12, 13] 
	0 refuels: []
Path 3:
	3 nodes: [12, 13, 18] 
	1 refuels: [(18, 163.0)]
Path 4:
	12 nodes: [25, 54, 53, 52, 24, 51, 23, 50, 19, 20, 18, 13] 
	3 refuels: [(51, 233.0), (19, 88), (18, 104)]
Path 5:
	11 nodes: [25, 54, 53, 52, 24, 51, 23, 50, 19, 20, 18] 
	3 refuels: [(51, 233.0), (19, 88), (18, 104)]
Path 6:
	17 nodes: [29, 28, 27, 68, 67, 22, 48, 47, 46, 45, 44, 43, 21, 42, 17, 18, 13] 
	4 refuels: [(67, 239.0), (44, 207), (17, 229), (18, 39)]
Path 7:
	16 nodes: [29, 28, 27, 68, 67, 22, 48, 47, 46, 45, 44, 43, 21, 42, 17, 18] 
	4 refuels: [(67, 239.0), (44, 207), (17, 229), (18, 39)]
Path 8:
	20 nodes: [31, 32, 34, 35, 38, 27, 68, 67, 22, 48, 47, 46, 45, 44, 43, 21, 42, 17, 18, 13] 
	5 refuels: [(34, 154.0), (67, 122), (44, 207), (17, 229), (18, 39)]
Path 9:
	19 nodes: [31, 32, 34, 35, 38, 27, 68, 67, 22, 48, 47, 46, 45, 44, 43, 21, 42, 17, 18] 
	5 ref

In [52]:
km = pd.DataFrame.from_dict(path_refuel_km, orient='index')
km.columns = ['fuel_km']

In [53]:
km.head()

,fuel_km
"(0, 18)",145.0
"(1, 18)",120.0
"(3, 18)",163.0
"(4, 51)",233.0
"(4, 19)",88.0


In [54]:
km['path_id'] = km.index.map(lambda x: int(x[0]))
km['refuel_at'] = km.index.map(lambda x: int(x[1]))
km = km[['path_id', 'refuel_at', 'fuel_km']].sort_values('path_id')

In [55]:
km.head()

,path_id,refuel_at,fuel_km
"(0, 18)",0,18,145.0
"(1, 18)",1,18,120.0
"(3, 18)",3,18,163.0
"(4, 51)",4,51,233.0
"(4, 19)",4,19,88.0


This table describes on each path, at each refueling point, how much mileage that refeuling point is responsible for covering.

In [56]:
km.to_csv(os.path.join(SCRATCH_FOLDER, 'Paths_fuel_km.csv'), index=False)